In [1]:
!nvidia-smi

Sat Jan  2 02:35:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import tensorflow as tf
import tensorflow_addons as tfa
from functools import partial

In [3]:
csv_path = '/content/drive/My Drive/project/mango/sample/label'
image_path = '/content/drive/My Drive/project/mango/sample/image/'

# csv_path = '/content/drive/My Drive/project/mango/train_dev/'
# image_path = '/content/drive/My Drive/project/mango/train_dev/Train/'

# csv_path = '/content/drive/My Drive/project/mango/train_dev/'
# image_path = '/content/drive/My Drive/project/mango/train_dev/Dev/'

# csv_path = '/content/drive/My Drive/project/mango/test/'
# image_path = '/content/drive/My Drive/project/mango/test/image/Test/'

In [4]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

def decode_image(image):
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = image/255
    # image = tf.reshape(image, [224, 224, 3])
    return image

def decode_label(label):
  label = tf.io.decode_raw(label, tf.float32)
  # label = tf.cast(label, tf.float32)
  return label

def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            'image_raw': tf.io.FixedLenFeature([], tf.string),
            'target': tf.io.FixedLenFeature([], tf.string),
        }
        if labeled
        else {"image_raw": tf.io.FixedLenFeature([], tf.string)}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image_raw'])
    if labeled:
        label = decode_label(example['target'])
        # label = tf.cast(label, tf.int32)
        return image, label
    return image

def load_dataset(filenames, labeled = False):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = True  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames)  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order)  
# uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled = labeled), num_parallel_calls = AUTOTUNE)
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

def get_dataset(filenames, labeled = False):
    dataset = load_dataset(filenames, labeled = labeled)
#     dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size = AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [15]:
def make_model():
    base_model = tf.keras.applications.ResNet50V2(
        input_shape=(224, 224, 3), include_top = False, weights = 'imagenet',
        classes = 5, pooling = 'None'
    )
    base_model.trainable = True

    inputs = tf.keras.layers.Input([224, 224, 3])
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # cla_dense = tf.keras.layers.Dense(32, activation = 'relu')(x)
    # loc_dense = tf.keras.layers.Dense(32, activation = 'relu')(x)
    cla_outputs = tf.keras.layers.Dense(5, activation = 'softmax')(x)
    loc_outputs = tf.keras.layers.Dense(4)(x)

    model = tf.keras.Model(inputs = inputs, outputs = loc_outputs)
    # model = tf.keras.Model(inputs = inputs, outputs = [cla_outputs, loc_outputs])
    f1_marco = tfa.metrics.F1Score(num_classes = 5, average = 'macro')
    model.compile(
        optimizer = tf.keras.optimizers.Adam(),
        # optimizer=tf.keras.optimizers.Adam(learning_rate = lr_schedule),
        loss = ['categorical_crossentropy', 'mse'],
        metrics = f1_marco
    )

    return model

In [6]:
train_dataset = get_dataset(os.path.join(csv_path, 'sample.tfrecord'), labeled = False)

In [7]:
train_dataset

<BatchDataset shapes: (None, None, None, None), types: tf.float32>

In [19]:
img = next(iter(train_dataset))

In [8]:
make_model().summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
resnet50v2 (Functional)         (None, 7, 7, 2048)   23564800    input_2[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 2048)         0           resnet50v2[0][0]                 
__________________________________________________________________________________________________
dense (Dense)                   (None, 5)            10245       global_average_pooling2d[0][0]   
______________________________________________________________________________________________

In [ ]:
my_model = make_model()
history = my_model.fit(
    train_dataset,
    epochs = 30,
)

In [16]:
my_model = make_model()
pre = my_model.predict(train_dataset)

In [13]:
len(pre)

107

In [17]:
pre

array([[ 0.868392  ,  0.3622032 ,  0.39059675,  0.47743812],
       [ 1.2765721 ,  0.00651523,  0.28149104, -0.3919453 ],
       [ 0.80796313,  0.41827235, -0.35058942,  0.35341096],
       [ 1.347966  ,  0.33056402,  0.65587544,  0.43488136],
       [ 0.394485  ,  0.09206256,  0.38113683,  0.26442426],
       [ 0.394485  ,  0.09206256,  0.38113683,  0.26442426],
       [ 0.86698145,  0.41532332,  0.01569355, -0.0349369 ],
       [-0.57505655,  0.35997084, -0.84502465, -0.38463324],
       [ 1.1478567 , -0.50381815, -1.2527709 ,  0.06523809],
       [ 1.3400598 , -0.05955404, -0.3653006 ,  0.95701873],
       [-0.12140727, -0.1923163 , -1.3053553 ,  1.608171  ],
       [-0.36336875,  0.24739784, -0.25366753, -0.45240217],
       [-0.36336875,  0.24739784, -0.25366753, -0.45240217],
       [ 1.9565924 , -0.6956794 , -0.7836295 , -0.98250544],
       [ 1.4079565 ,  0.97311354, -0.18288195,  0.41879526],
       [ 1.9272155 , -1.0460355 , -0.78622556,  0.6198224 ],
       [ 1.3428802 , -1.